<a href="https://colab.research.google.com/github/arrowko/daily-leaderboard/blob/main/kapitola_2_BezUcitela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kapitola 2: IRIS ako prvá praktická ukážka klastrovania

V tejto kapitole sa naučíme aplikovať klastrovanie na reálny dataset kvetov IRIS. Zoznámime sa s teoretickými základmi klastrovania, naučíme sa, ako funguje K-means algoritmus, a naučíme sa vizualizovať a interpretovať výsledky klastrovania.

---

## ⚠️ Dôležité pred začatím

**Všetky úlohy a otázky z tejto kapitoly vypracujte do JEDNÉHO laboratórneho referátu v PDF formáte.**

Referát musí obsahovať:
- **Odpovede na všetky otázky na rozmyslenie** z jednotlivých podkapitol
- **Všetky praktické úlohy** s kódom, výsledkami a grafy
- **Popisy kódov** - vysvetlenie, čo robí každý kód krok za krokom
- **Pozorovania a závery** - čo ste sa naučili, aké problémy ste mali

**Formát odovzdania:**
- PDF dokument: `Kapitola2_Ulohy_VaseMeno.pdf`
- Python súbory (ak sú súčasťou úlohy): `uloha_X_Y.py`, atď.

**Viac informácií o formáte a štruktúre referátu nájdete v sekcii 2.9 na konci kapitoly.**

## 2.1 Úvod do IRIS datasetu a problematika bez labelov

**Pripomenutie:** V predchádzajúcich lekciách sme pracovali s dátami, kde sme vždy mali správne odpovede (labely). V datasetu MNIST sme vedeli, že obrázok predstavuje číslo 0, 1, 2, atď.

Teraz sa pozrieme na dataset IRIS - dataset kvetov, kde máme merania kvetov (dĺžka a šírka okvetných lístkov a kalichov), ale **nevieme, aký druh kvetu to je**.

**Predstavte si situáciu:** Máte 150 kvetov, každý má 4 merania (napr. dĺžka okvetného lístka), ale nikto vám nepovedal, koľko druhov kvetov existuje ani ktorý kvet patrí ku ktorému druhu. Ako by ste zistili, koľko druhov existuje? Počítač to dokáže pomocou klastrovania!

### 2.1.1 Načítanie a základná explorácia

In [ ]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

# Načítanie datasetu IRIS
url = "https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv"
df = pl.read_csv(url)

# Základná explorácia
print(f"Počet vzorov: {len(df)}")
print(f"Počet vlastností: {len(df.columns) - 1}")  # -1 pretože species je label
print("\nPrvých 5 riadkov:")
print(df.head())
print("\nZákladné štatistiky:")
print(df.describe())

### 💭 Otázky na rozmyslenie (2.1.1)

1. Koľko vzorov máme? Koľko vlastností?
2. Aké vlastnosti máme? Čo predstavujú (sepal length, sepal width, petal length, petal width)?
3. Ako sa líši tento dataset od MNIST? (Tabuľkové dáta vs. obrázky, máme stĺpec "species", ale ten použijeme len na kontrolu)

**Zapíšte si svoje odpovede.**

### 2.1.2 Vizuálna explorácia dát

Predtým, ako použijeme algoritmus, pozrime sa na dáta vizuálne. Môžeme vidieť nejaké skupiny?

In [ ]:
# Pripravíme dáta pre vizualizáciu
# Extrahujeme hodnoty do numpy polí
petal_length = df['petal_length'].to_numpy()
petal_width = df['petal_width'].to_numpy()
sepal_length = df['sepal_length'].to_numpy()
sepal_width = df['sepal_width'].to_numpy()
species = df['species'].to_numpy()

# Vytvoríme číselné kódy pre druhy (pre farby v grafoch)
species_unique = df['species'].unique().to_numpy()
species_numeric = np.array([np.where(species_unique == s)[0][0] for s in species])

# Vytvoríme scatter ploty pre rôzne kombinácie vlastností
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Petal length vs Petal width (najjasnejšie oddelené)
axes[0, 0].scatter(petal_length, petal_width,
                   c=species_numeric, cmap='viridis', alpha=0.6, s=50)
axes[0, 0].set_xlabel('Petal Length (cm)')
axes[0, 0].set_ylabel('Petal Width (cm)')
axes[0, 0].set_title('Petal Length vs Petal Width\n(farba = skutočný druh)')
axes[0, 0].grid(True, alpha=0.3)

# Sepal length vs Sepal width
axes[0, 1].scatter(sepal_length, sepal_width,
                   c=species_numeric, cmap='viridis', alpha=0.6, s=50)
axes[0, 1].set_xlabel('Sepal Length (cm)')
axes[0, 1].set_ylabel('Sepal Width (cm)')
axes[0, 1].set_title('Sepal Length vs Sepal Width\n(farba = skutočný druh)')
axes[0, 1].grid(True, alpha=0.3)

# Petal length vs Sepal length
axes[1, 0].scatter(petal_length, sepal_length,
                   c=species_numeric, cmap='viridis', alpha=0.6, s=50)
axes[1, 0].set_xlabel('Petal Length (cm)')
axes[1, 0].set_ylabel('Sepal Length (cm)')
axes[1, 0].set_title('Petal Length vs Sepal Length\n(farba = skutočný druh)')
axes[1, 0].grid(True, alpha=0.3)

# Všetky vlastnosti (PCA - len úvod, detailne v kapitole 5)
from sklearn.decomposition import PCA
X_all = df.select(['sepal_length', 'sepal_width', 'petal_length', 'petal_width']).to_numpy()
pca = PCA(n_components=2)
X_2d = pca.fit_transform(X_all)
axes[1, 1].scatter(X_2d[:, 0], X_2d[:, 1],
                   c=species_numeric, cmap='viridis', alpha=0.6, s=50)
axes[1, 1].set_xlabel(f'Prvá hlavná komponenta ({pca.explained_variance_ratio_[0]:.1%})')
axes[1, 1].set_ylabel(f'Druhá hlavná komponenta ({pca.explained_variance_ratio_[1]:.1%})')
axes[1, 1].set_title('Všetky 4 vlastnosti (PCA)\n(farba = skutočný druh)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Predict:**

Predtým, ako pokračujeme, zamyslite sa:
- Koľko skupín (druhov) vidíte v grafoch?
- Ktorá kombinácia vlastností najlepšie oddeluje skupiny?
- Ako by ste manuálne rozdelili kvety do skupín?

### 💭 Otázky na rozmyslenie (2.1.2)

1. Vidíte v grafoch nejaké skupiny? Koľko ich je?
2. Ktorá kombinácia vlastností (napr. petal length vs. petal width) najlepšie oddeluje skupiny?
3. V MNIST sme mali labely (čísla 0-9). Tu máme stĺpec "species", ale povedzme, že ho nemáme. Ako by sme zistili, koľko druhov existuje?

**Zapíšte si svoje odpovede.**

## 2.2 Čo je klastrovanie? Teoretické základy

**Analógia:** Predstavte si, že máte hromadu oblečenia a chcete ho zoradiť do šatníka. Nemáte nápisy "tričká", "nohavice", "mikiny", ale musíte sami zistiť, čo patrí k sebe. Podobné veci (napr. všetky tričká) dáte do jedného šuflíka, nohavice do druhého, atď. To je presne to, čo robí klastrovanie - automaticky nájde skupiny podobných vecí.

Klastrovanie (clustering) je typ učenia bez učiteľa, ktorého cieľom je **automaticky nájsť skupiny podobných objektov** v dátach.

**Pripomenutie rozdielu:** Na rozdiel od klasifikácie (učenie s učiteľom), kde máme labely a učíme sa predpovedať, do ktorej triedy objekt patrí, v klastrovaní **nemáme žiadne labely** - musíme sami objaviť, aké skupiny existujú.

### 2.2.1 Definícia a účel klastrovania

#### Porovnanie: Klasifikácia vs. Klastrovanie

**Pripomenutie:** V kapitole 1 sme si vysvetlili rozdiel medzi učením s učiteľom a bez učiteľa. Teraz to aplikujeme na konkrétny príklad:

| Aspekt | Klasifikácia (s učiteľom) | Klastrovanie (bez učiteľa) |
|--------|-------------------------|---------------------------|
| **Vstupné dáta** | Vlastnosti + labely | Len vlastnosti |
| **Cieľ** | Predpovedať label pre nový objekt | Nájsť skupiny podobných objektov |
| **Vieme vopred** | Koľko tried existuje, aké sú | Nič - musíme objaviť |
| **Príklad** | Rozpoznať, či je to pes alebo mačka (viete, že existujú psy a mačky) | Zoskupiť podobné obrázky zvierat (neviete, koľko druhov existuje) |
| **Validácia** | Porovnanie s pravdivými labelmi | Nemáme pravdivé labely - len vizuálna kontrola |

**Kľúčová myšlienka:** Klastrovanie je založené na myšlienke **podobnosti**, ktorú sme si vysvetlili v kapitole 1. Objekty, ktoré sú si podobné (majú podobné vlastnosti), by mali byť v rovnakej skupine. Objekty, ktoré sú odlišné, by mali byť v rôznych skupinách.

**Jednoducho povedané:**
- **Klasifikácia:** Máte nápisy a učíte sa ich rozpoznávať
- **Klastrovanie:** Nemáte nápisy, ale sami zistíte, koľko druhov existuje

### 2.2.2 Kedy používame klastrovanie?

Klastrovanie používame v situáciách, kde:
- Nemáme labely (nevieme, do ktorej skupiny objekt patrí)
- Chceme objaviť skryté vzory v dátach
- Chceme zorganizovať dáta do skupín pre ďalšiu analýzu

#### Príklady zo života:

1. **Segmentácia zákazníkov:**
   - **Situácia:** Máte obchod a chcete rozdeliť zákazníkov do skupín podľa ich nákupného správania
   - **Problém:** Neviete, koľko skupín existuje ani ktorí zákazníci patria do ktorej skupiny
   - **Riešenie:** Klastrovanie automaticky nájde skupiny (napr. "mladí aktívni zákazníci", "starší zákazníci")

2. **Organizovanie dokumentov:**
   - **Situácia:** Máte 1000 článkov a chcete ich zoradiť podľa témy
   - **Problém:** Neviete, koľko tém existuje
   - **Riešenie:** Klastrovanie zoskupí podobné články (napr. všetky články o športe spolu)

3. **Fotky z dovolenky:**
   - **Situácia:** Máte 500 fotiek z dovolenky a chcete ich zoradiť do albumov
   - **Problém:** Neviete, koľko albumov potrebujete
   - **Riešenie:** Klastrovanie zoskupí podobné fotky (napr. všetky fotky z pláže spolu, všetky fotky z hôr spolu)

### 💭 Otázky na rozmyslenie (2.2.1)

1. V akých situáciách by bolo užitočné použiť klastrovanie?
2. V čom sa líši klastrovanie od klasifikácie, ktorú sme robili v PyTorch?
3. Prečo nemôžeme použiť klasifikáciu, keď nemáme labely?

**Zapíšte si svoje odpovede.**

### 2.2.3 Základné princípy klastrovania

**Pripomenutie z kapitoly 1:** Všetky algoritmy klastrovania sú založené na myšlienke podobnosti a vzdialenosti, ktorú sme si už vysvetlili.

Všetky algoritmy klastrovania sú založené na dvoch základných princípoch:

**1. Podobnosť (Similarity):**
- Objekty v rovnakej skupine by mali byť si podobné
- **Jednoducho povedané:** Podobné veci patria spolu (ako tričká do jedného šuflíka)
- Meriame pomocou vzdialenosti (napr. Euklidovská vzdialenosť, ktorú sme už videli v kapitole 1)

**2. Separácia (Separation):**
- Objekty z rôznych skupín by mali byť odlišné
- Skupiny by mali byť od seba oddelené
- **Jednoducho povedané:** Rôzne veci by mali byť v rôznych skupinách (tričká a nohavice nie sú spolu)

In [ ]:
# Vizuálna ilustrácia princípov klastrovania
np.random.seed(42)

# Vytvoríme dva príklady: dobre oddelené vs. prekrývajúce sa skupiny
skupina1 = np.random.normal([2, 2], 0.3, (20, 2))
skupina2_dobre = np.random.normal([5, 5], 0.3, (20, 2))
skupina2_close = np.random.normal([3, 3], 0.3, (20, 2))

plt.figure(figsize=(12, 5))

# Ľavý graf: Dobre oddelené skupiny
plt.subplot(121)
plt.scatter(skupina1[:, 0], skupina1[:, 1], c='blue', alpha=0.6, label='Skupina 1', s=50)
plt.scatter(skupina2_dobre[:, 0], skupina2_dobre[:, 1], c='red', alpha=0.6, label='Skupina 2', s=50)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Dobre oddelené skupiny\n(ľahké pre algoritmus)')
plt.legend()
plt.grid(True, alpha=0.3)

# Pravý graf: Prekrývajúce sa skupiny
plt.subplot(122)
plt.scatter(skupina1[:, 0], skupina1[:, 1], c='blue', alpha=0.6, label='Skupina 1', s=50)
plt.scatter(skupina2_close[:, 0], skupina2_close[:, 1], c='red', alpha=0.6, label='Skupina 2', s=50)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Prekrývajúce sa skupiny\n(ťažšie pre algoritmus)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Pozorovania:")
print("- V ľavom grafe sú skupiny jasne oddelené - algoritmus ich nájde ľahko")
print("- V pravom grafe sa skupiny prekrývajú - algoritmus môže robiť chyby")
print("- Čím lepšie oddelené skupiny, tým lepšie výsledky klastrovania")

### 💭 Otázky na rozmyslenie (2.2.2)

1. Prečo je ľahšie nájsť skupiny v ľavom grafe ako v pravom?
2. Čo by sa stalo, keby skupiny boli ešte viac prekrývajúce sa?
3. Ako by ste definovali "podobnosť" medzi dvoma kvetmi IRIS?

**Zapíšte si svoje odpovede.**

## 2.3 Ako funguje K-means algoritmus? Teoretické základy

**Analógia:** Predstavte si, že máte 30 žiakov a 3 učiteľov. Chcete rozdeliť žiakov do 3 skupín tak, aby každá skupina bola okolo jedného učiteľa. Najprv náhodne rozdelíte žiakov. Potom každý učiteľ vypočíta, kde je "stred" jeho skupiny (kde by sa všetci žiaci stretli). Potom sa žiaci presunú k najbližšiemu učiteli. Toto sa opakuje, až kým sa žiaci prestanú presúvať. To je presne to, čo robí K-means!

K-means je jeden z najpopulárnejších algoritmov klastrovania. Jeho názov pochádza z toho, že hľadá **K skupín** (klastrov) a používa **stredy skupín** (means) na ich reprezentáciu.

**Prečo sa to volá K-means?**
- **K** = počet skupín, ktoré chceme nájsť (napr. 3)
- **means** = stredy skupín (každá skupina má "stred", okolo ktorého sú objekty)

### 2.3.1 Úvod do K-means

**Základné informácie:**
- **Vstup:** Dáta (bez labelov) + počet skupín K (napr. 3)
- **Výstup:** Každý objekt je priradený do jednej zo K skupín
- **Predpoklady:** Skupiny sú približne kruhové a oddelené (ako naše kvety IRIS)

**Jednoducho povedané:** Dáte algoritmu dáta a povediete mu, koľko skupín má nájsť. Algoritmus automaticky nájde skupiny a každý objekt priradí do jednej skupiny.

### 2.3.2 Ako funguje K-means? Krok za krokom

K-means opakuje niekoľko krokov, až kým sa výsledok prestane meniť. Pozrime sa na to krok za krokom:

**Krok 1: Začneme s náhodnými miestami (inicializácia)**
- Predstavte si, že máte mapu a náhodne označíte 3 miesta (napr. pomocou špendlíka)
- Tieto miesta sa nazývajú **centroidy** (stredy skupín) - sú to "stredy", okolo ktorých budú skupiny
- **Jednoducho povedané:** Náhodne vyberieme 3 miesta, kde by mohli byť skupiny
- Zatiaľ nevieme, kde sú skutočné skupiny - len hádame

**Krok 2: Každý objekt sa presunie k najbližšiemu miestu (priradenie)**
- Pre každý kvet vypočítame vzdialenosť ku všetkým 3 miestam
- **Pripomenutie:** Vzdialenosť počítame rovnako ako v kapitole 1 (Euklidovská vzdialenosť)
- Kvet sa "presunie" k najbližšiemu miestu
- Teraz máme 3 skupiny kvetov (každá okolo jedného miesta)

**Krok 3: Vypočítame nové stredy skupín (aktualizácia)**
- Pre každú skupinu vypočítame, kde je jej "stred" (priemer všetkých kvetov v skupine)
- **Analógia:** Je to ako keby sme vypočítali, kde by sa všetci žiaci v skupine stretli
- Toto je nové miesto, okolo ktorého sú kvety

**Krok 4: Opakujeme**
- Vrátime sa na krok 2 - kvety sa znova presunú k najbližším miestam
- Opakujeme kroky 2 a 3, až kým sa miesta prestanú meniť
- Keď sa miesta prestanú meniť, máme výsledok!

**Jednoducho povedané:** Opakujeme presúvanie kvetov a výpočet nových stredov, až kým sa nič neprestane meniť.

In [ ]:
# Vizuálna ilustrácia krokov K-means (statická verzia - ukážeme niekoľko iterácií)
from sklearn.cluster import KMeans

# Vytvoríme jednoduché dáta
np.random.seed(42)
data = np.vstack([
    np.random.normal([2, 2], 0.5, (30, 2)),
    np.random.normal([6, 6], 0.5, (30, 2)),
    np.random.normal([2, 6], 0.5, (30, 2))
])

# Ukážeme niekoľko iterácií
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for iter_num in range(6):
    row = iter_num // 3
    col = iter_num % 3

    # Vytvoríme K-means s max_iter=iter_num+1
    kmeans = KMeans(n_clusters=3, max_iter=iter_num+1, random_state=42, n_init=1, init='random')
    kmeans.fit(data)
    labels = kmeans.predict(data)
    centers = kmeans.cluster_centers_

    # Vykreslíme
    colors = ['blue', 'green', 'orange']
    for i in range(3):
        mask = labels == i
        axes[row, col].scatter(data[mask, 0], data[mask, 1], c=colors[i], alpha=0.6, s=50)
    axes[row, col].scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=200, linewidths=2)
    axes[row, col].set_title(f'Iterácia {iter_num+1}')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Pozorovania:")
print("- V každej iterácii sa centroidy (červené X) posúvajú")
print("- Body sa priraďujú k najbližšiemu centroidu")
print("- Keď sa centroidy prestanú meniť, algoritmus konvergoval")

### 💭 Otázky na rozmyslenie (2.3.1)

1. Prečo sa stredy skupín (centroidy) menia v každej iterácii? (Pomôcka: Pozrite sa na grafy vyššie)
2. Čo sa stane, keď sa stredy skupín prestanú meniť? (Pomôcka: Algoritmus sa zastaví)
3. Prečo je dôležité, aby sme správne začali (inicializovali)? (Pomôcka: Čo by sa stalo, keby sme všetky 3 stredy umiestnili na jedno miesto?)

**Zapíšte si svoje odpovede.**

### 2.3.3 Prečo to funguje? Matematický základ

**Jednoducho povedané:** K-means chce, aby všetky kvety v skupine boli čo najbližšie k stredu skupiny. Čím bližšie sú kvety k stredu, tým lepšie je zoskupenie.

**Technicky:** K-means sa snaží minimalizovať **vnútornú vzdialenosť skupín** (inertia alebo WCSS - Within-Cluster Sum of Squares).

**Čo to znamená?**
- Pre každú skupinu vypočítame vzdialenosti všetkých kvetov od stredu skupiny
- Algoritmus sa snaží, aby tieto vzdialenosti boli čo najmenšie
- **Analógia:** Je to ako keby sme chceli, aby všetci žiaci v skupine boli čo najbližšie k svojmu učiteli

**Prečo to robíme?**
- Ak sú kvety blízko k stredu skupiny, znamená to, že sú si podobné
- Ak sú kvety ďaleko od stredu, možno nepatria do tejto skupiny
- Dobré zoskupenie = krátke vzdialenosti od stredu

In [ ]:
# Ukážeme, čo znamená "minimalizovať vzdialenosť"
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(data)
labels = kmeans.predict(data)
centers = kmeans.cluster_centers_

plt.figure(figsize=(10, 10))
colors = ['blue', 'green', 'orange']
for i in range(3):
    mask = labels == i
    plt.scatter(data[mask, 0], data[mask, 1], c=colors[i], alpha=0.6, s=50, label=f'Skupina {i+1}')

    # Vykreslíme vzdialenosti od centroidu (len prvých 5 pre prehľadnosť)
    for point in data[mask][:5]:
        plt.plot([point[0], centers[i, 0]], [point[1], centers[i, 1]],
                'k--', alpha=0.3, linewidth=1)

plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=200,
           linewidths=2, label='Centroidy')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Vzdialenosti bodov od centroidov\n(čím kratšie, tým lepšie)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Inertia (WCSS): {kmeans.inertia_:.2f}")
print("Čím nižšia hodnota, tým lepšie zoskupené dáta")
print("Algoritmus sa snaží túto hodnotu minimalizovať")

### 💭 Otázky na rozmyslenie (2.3.2)

1. Čo znamená "minimalizovať vzdialenosť"? (Pomôcka: Pozrite sa na graf vyššie - čím kratšie čiary, tým lepšie)
2. Prečo chceme, aby kvety boli čo najbližšie k stredu skupiny? (Pomôcka: Čo to znamená, keď sú kvety blízko seba?)
3. Čo by sa stalo, keby sme mali veľmi vysokú hodnotu inertia? (Pomôcka: Znamenalo by to, že kvety sú ďaleko od stredov skupín)

**Zapíšte si svoje odpovede.**

### 2.3.4 Kde K-means nefunguje? Limitácie

K-means má niekoľko limitácií, ktoré je dôležité poznať:

**1. Musíme zadať počet skupín K**
- **Problém:** V praxi často nevieme, koľko skupín existuje
- **Riešenie:** Musíme experimentovať alebo použiť iné metódy (napr. elbow metóda, ktorú uvidíme neskôr)
- **Príklad:** Ak nevieme, koľko druhov kvetov existuje, musíme skúšať rôzne počty (2, 3, 4, 5...)

**2. Predpokladá kruhové skupiny**
- **Problém:** K-means funguje dobre len keď sú skupiny približne kruhové
- **Analógia:** Predstavte si, že máte skupiny tvaru:
  - ✅ **Kruh** - K-means to zvládne (ako naše kvety IRIS)
  - ❌ **Polmesiac** - K-means to nezvládne (rozdeli to na dve skupiny namiesto jednej)
- **Prečo?** K-means hľadá "stredy" skupín. Ak máte skupinu tvaru polmesiaca, stred by bol mimo skupiny, čo nedáva zmysel

**3. Citlivý na náhodný začiatok (inicializácia)**
- **Problém:** Rôzne náhodné začiatky môžu viesť k rôznym výsledkom
- **Analógia:** Ak začneme s rôznymi náhodnými miestami na mape, môžeme skončiť s rôznymi skupinami
- **Riešenie:** Spustiť viackrát a vybrať najlepší výsledok (preto používame n_init=10)

**4. Citlivý na nezvyčajné objekty (outliers)**
- **Problém:** Nezvyčajné objekty (napr. kvet, ktorý je úplne iný) môžu ovplyvniť pozíciu stredov skupín
- **Príklad:** Ak máte jeden kvet, ktorý je 10-krát väčší ako ostatné, môže "potiahnuť" stred skupiny k sebe

In [ ]:
# Vizuálne príklady problémov
from sklearn.datasets import make_moons

# Príklad 1: Nekruhové skupiny
moons, _ = make_moons(n_samples=200, noise=0.1, random_state=42)

kmeans_moons = KMeans(n_clusters=2, random_state=42)
labels_moons = kmeans_moons.fit_predict(moons)

plt.figure(figsize=(14, 5))

plt.subplot(121)
plt.scatter(moons[:, 0], moons[:, 1], c=labels_moons, cmap='viridis', alpha=0.6, s=50)
plt.scatter(kmeans_moons.cluster_centers_[:, 0], kmeans_moons.cluster_centers_[:, 1],
           c='red', marker='X', s=200, linewidths=2)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('K-means na nekrehové skupiny\n(nefunguje dobre)')
plt.grid(True, alpha=0.3)

# Príklad 2: Rôzne inicializácie
plt.subplot(122)
for init in range(3):
    kmeans_init = KMeans(n_clusters=3, random_state=init, n_init=1, init='random')
    kmeans_init.fit(data)
    labels_init = kmeans_init.predict(data)
    centers_init = kmeans_init.cluster_centers_

    colors_init = ['blue', 'green', 'orange']
    markers = ['o', 's', '^']
    for i in range(3):
        mask = labels_init == i
        plt.scatter(data[mask, 0], data[mask, 1], c=colors_init[i],
                   alpha=0.3, s=30, marker=markers[init])
    plt.scatter(centers_init[:, 0], centers_init[:, 1],
               c='red', marker='X', s=100, linewidths=1, alpha=0.5)

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Rôzne inicializácie\n(rôzne výsledky)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Pozorovania:")
print("- K-means nefunguje dobre na nekrehové skupiny (ľavý graf)")
print("- Rôzne inicializácie vedú k rôznym výsledkom (pravý graf)")
print("- Preto používame n_init=10 (spustí 10-krát a vyberie najlepší)")

### 💭 Otázky na rozmyslenie (2.3.3)

1. Prečo K-means nefunguje dobre na nekrehové skupiny? (Pomôcka: Pozrite sa na graf s polmesiacom vyššie - čo by sa stalo so stredom skupiny?)
2. Čo by ste robili, keby ste nevedeli, koľko skupín existuje? (Pomôcka: Museli by ste skúšať rôzne počty a pozorovať výsledky)
3. Ako by ste riešili problém s rôznymi náhodnými začiatkami? (Pomôcka: Spustili by ste algoritmus viackrát a vybrali najlepší výsledok)

**Zapíšte si svoje odpovede.**

## 2.4 Praktická aplikácia: K-means na IRIS

Teraz aplikujeme K-means na IRIS dataset. **Dôležité:** Použijeme len vlastnosti (sepal length, sepal width, petal length, petal width), **nie** stĺpec species. Species použijeme len na kontrolu výsledkov (aby sme videli, ako dobre algoritmus funguje).

**Analógia:** Je to ako keby sme mali kvety s nápismi (species), ale zakryli sme ich a nechali algoritmus, aby sám zistil, koľko druhov existuje. Potom odhalíme nápisy a pozrieme sa, či algoritmus trafil správne.

### 2.4.1 Prvé klastrovanie

In [ ]:
from sklearn.cluster import KMeans

# DÔLEŽITÉ: Extrahovanie vlastností BEZ species!
# Použijeme len merania kvetov, nie ich druhy
# Species použijeme len neskôr na kontrolu výsledkov
X = df.select(['sepal_length', 'sepal_width', 'petal_length', 'petal_width']).to_numpy()

# K-means s 3 skupinami
# n_clusters=3: Hľadáme 3 skupiny (pretože vieme, že sú 3 druhy)
# random_state=42: Náhodný začiatok (pre reprodukovateľnosť výsledkov)
# n_init=10: Spustí algoritmus 10-krát s rôznymi začiatkami a vyberie najlepší výsledok
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)  # Každý kvet dostane číslo skupiny (0, 1 alebo 2)

print(f"Počet skupín: {len(np.unique(clusters))}")
print(f"Inertia (WCSS): {kmeans.inertia_:.2f}")
print("(Čím nižšia hodnota, tým lepšie zoskupené dáta)")
print(f"\nCentroidy (stredy skupín):")
print("(Toto sú 'stredy' skupín - priemerné hodnoty vlastností pre každú skupinu)")
for i, center in enumerate(kmeans.cluster_centers_):
    print(f"  Skupina {i}: sepal_length={center[0]:.2f}, sepal_width={center[1]:.2f}, "
          f"petal_length={center[2]:.2f}, petal_width={center[3]:.2f}")

**PRIMM - Predict:**

Pred spustením kódu zamyslite sa:
- Čo si myslíte, že sa stane?
- Nájde algoritmus 3 skupiny? (Pomôcka: Nastavili sme K=3)
- Budú skupiny zodpovedať skutočným druhom? (Pomôcka: Pozrite sa na grafy z 2.1 - videli ste tam 3 skupiny?)

**Zapíšte si svoje predpovede pred spustením kódu.**

### 2.4.2 Vizualizácia výsledkov

In [ ]:
# Vizualizácia klastrov
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Petal length vs Petal width - s klastrami
axes[0, 0].scatter(X[:, 2], X[:, 3], c=clusters, cmap='viridis', alpha=0.6, s=50)
axes[0, 0].scatter(kmeans.cluster_centers_[:, 2], kmeans.cluster_centers_[:, 3],
                  c='red', marker='X', s=200, linewidths=2, label='Centroidy')
axes[0, 0].set_xlabel('Petal Length (cm)')
axes[0, 0].set_ylabel('Petal Width (cm)')
axes[0, 0].set_title('Klastrovanie\n(farba = nájdená skupina)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Petal length vs Petal width - so skutočnými druhmi
axes[0, 1].scatter(X[:, 2], X[:, 3],
                   c=species_numeric, cmap='viridis', alpha=0.6, s=50)
axes[0, 1].set_xlabel('Petal Length (cm)')
axes[0, 1].set_ylabel('Petal Width (cm)')
axes[0, 1].set_title('Skutočné druhy\n(farba = skutočný druh)')
axes[0, 1].grid(True, alpha=0.3)

# Sepal length vs Sepal width - s klastrami
axes[1, 0].scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', alpha=0.6, s=50)
axes[1, 0].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                  c='red', marker='X', s=200, linewidths=2, label='Centroidy')
axes[1, 0].set_xlabel('Sepal Length (cm)')
axes[1, 0].set_ylabel('Sepal Width (cm)')
axes[1, 0].set_title('Klastrovanie\n(farba = nájdená skupina)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Sepal length vs Sepal width - so skutočnými druhmi
axes[1, 1].scatter(X[:, 0], X[:, 1],
                  c=species_numeric, cmap='viridis', alpha=0.6, s=50)
axes[1, 1].set_xlabel('Sepal Length (cm)')
axes[1, 1].set_ylabel('Sepal Width (cm)')
axes[1, 1].set_title('Skutočné druhy\n(farba = skutočný druh)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Investigate:**

Pozrite sa na grafy a porovnajte ľavé grafy (nájdené skupiny) s pravými grafmi (skutočné druhy):
- Sú skupiny jasne oddelené? (Pomôcka: Pozrite sa na petal grafy - sú tam jasné skupiny?)
- Ktorá kombinácia vlastností najlepšie oddeluje skupiny? (Pomôcka: Porovnajte petal vs. sepal grafy)
- Ako dobre zodpovedajú nájdené skupiny skutočným druhom? (Pomôcka: Porovnajte farby v ľavých a pravých grafoch)

**Zapíšte si svoje pozorovania.**

### 💭 Otázky na rozmyslenie (2.4.1)

1. Vidíte v grafoch jasné skupiny?
2. Ktorá kombinácia vlastností (petal alebo sepal) lepšie oddeluje skupiny?
3. Prečo sa skupiny v sepal grafoch viac prekrývajú?

**Zapíšte si svoje odpovede.**

## 2.5 Porovnanie s reálnymi druhmi a validácia

Teraz porovnáme výsledky klastrovania so skutočnými druhmi. **Pamätajte:** Species sme nepoužili pri učení - používame ho len na kontrolu, aby sme videli, ako dobre algoritmus funguje.

### 2.5.1 Porovnanie klastrov so skutočnými druhmi

In [ ]:
from sklearn.metrics import confusion_matrix, adjusted_rand_score
import seaborn as sns

# Porovnanie nájdených skupín so skutočnými druhmi
# DÔLEŽITÉ: Species sme nepoužili pri učení - používame ho len na kontrolu!
# Confusion matrix ukazuje, koľko kvetov z každého druhu bolo priradených do každej skupiny
cm = confusion_matrix(species_numeric, clusters)

# Vizualizácia confusion matrix
plt.figure(figsize=(12, 5))

plt.subplot(121)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Klastr 0', 'Klastr 1', 'Klastr 2'],
           yticklabels=df['species'].unique().to_numpy())
plt.xlabel('Nájdené skupiny (klastry)')
plt.ylabel('Skutočné druhy')
plt.title('Confusion Matrix\n(ako dobre algoritmus trafil)')

# Adjusted Rand Index (ARI) - metrika podobnosti
ari = adjusted_rand_score(species_numeric, clusters)
print(f"Adjusted Rand Index (ARI): {ari:.3f}")
print("ARI = 1.0 znamená perfektné zhodu")
print("ARI = 0.0 znamená náhodné priradenie")
print(f"\nNáš výsledok: {ari:.3f} - algoritmus našiel skupiny, ktoré približne zodpovedajú skutočným druhom")

# Porovnanie v tabuľke
comparison_df = pl.DataFrame({
    'Index': range(len(df)),
    'Skutočný druh': df['species'].to_numpy(),
    'Nájdená skupina': clusters
})

# Spočítame, koľko kvetov z každého druhu je v každej skupine
pivot = df.select(['species']).with_columns(pl.Series('cluster', clusters)).group_by(['species', 'cluster']).agg(pl.count().alias('count'))
print("\nRozdelenie kvetov:")
print(pivot)

# Vykreslíme rozdelenie
plt.subplot(122)
# Vytvoríme pivot tabuľku pomocou polars
pivot_data = df.select(['species']).with_columns(pl.Series('cluster', clusters))
pivot_counts = pivot_data.group_by(['species', 'cluster']).agg(pl.count().alias('count'))
# Pre konverziu na matplotlib potrebujeme numpy array
species_list = df['species'].unique().to_numpy()
cluster_counts = []
for species in species_list:
    species_data = pivot_counts.filter(pl.col('species') == species)
    counts = [0, 0, 0]
    for row in species_data.iter_rows(named=True):
        counts[row['cluster']] = row['count']
    cluster_counts.append(counts)
cluster_counts = np.array(cluster_counts)
# Vykreslíme
x = np.arange(len(species_list))
width = 0.25
for i in range(3):
    plt.bar(x + i*width, cluster_counts[:, i], width, label=f'Skupina {i}')
plt.xlabel('Skutočný druh')
plt.ylabel('Počet kvetov')
plt.title('Rozdelenie kvetov do skupín')
plt.xticks(x + width, species_list, rotation=45)
plt.legend(title='Nájdená skupina')
plt.tight_layout()
plt.show()

**Investigate:**

Analyzujte výsledky:
- Koľko kvetov bolo správne zoskupených? (Pomôcka: Pozrite sa na confusion matrix - väčšie čísla na diagonále = lepšie)
- Ktoré druhy boli najťažšie rozlíšiť? (Pomôcka: Pozrite sa, kde sú najväčšie chyby v confusion matrix)
- Prečo sa niektoré kvety zoskupili nesprávne? (Pomôcka: Pozrite sa na graf s chybami - kde sa nachádzajú?)

**Zapíšte si svoje pozorovania.**

### 💭 Otázky na rozmyslenie (2.5.1)

1. Ako dobre algoritmus "trafil" skutočné druhy?
2. Ktoré druhy boli najťažšie rozlíšiť? Prečo?
3. Čo to znamená, keď algoritmus nájde skupiny, ktoré nezodpovedajú presne skutočným druhom?

**Zapíšte si svoje odpovede.**

### 2.5.2 Interpretácia chýb

Nie všetky kvety boli správne zoskupené. Pozrime sa, prečo:

In [ ]:
# Nájdeme kvety, ktoré boli nesprávne zoskupené
errors = df.with_columns([
    pl.Series('cluster', clusters),
    pl.Series('species_code', species_numeric)
])

# Zobrazíme príklady chýb
print("Príklady nesprávne zoskupených kvetov:")
error_mask = errors['species_code'] != errors['cluster']
print(errors.filter(error_mask).head(10))

# Vykreslíme chyby
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Všetky kvety
axes[0].scatter(X[:, 2], X[:, 3], c=clusters, cmap='viridis', alpha=0.6, s=50)
axes[0].set_xlabel('Petal Length (cm)')
axes[0].set_ylabel('Petal Width (cm)')
axes[0].set_title('Všetky kvety\n(farba = nájdená skupina)')
axes[0].grid(True, alpha=0.3)

# Zvýrazníme chyby
error_mask_array = species_numeric != clusters
axes[1].scatter(X[~error_mask_array, 2], X[~error_mask_array, 3], c=clusters[~error_mask_array],
               cmap='viridis', alpha=0.3, s=50, label='Správne')
axes[1].scatter(X[error_mask_array, 2], X[error_mask_array, 3], c='red',
               marker='X', s=200, linewidths=2, label='Chyby')
axes[1].set_xlabel('Petal Length (cm)')
axes[1].set_ylabel('Petal Width (cm)')
axes[1].set_title('Chyby v klastrovaní')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPočet chýb: {np.sum(error_mask_array)} z {len(df)} kvetov")
print(f"Presnosť: {(1 - np.sum(error_mask_array)/len(df))*100:.1f}%")

### 💭 Otázky na rozmyslenie (2.5.2)

1. Prečo sa niektoré kvety zoskupili nesprávne? (Pomôcka: Pozrite sa na graf - kde sú chyby? Sú na hraniciach skupín?)
2. Kde sa nachádzajú chyby v grafe? (Pomôcka: Sú chyby na hraniciach skupín, alebo všade?)
3. Čo by sa stalo, keby sme mali viac vlastností alebo iné vlastnosti? (Pomôcka: Viac informácií = lepšie alebo horšie výsledky?)

**Zapíšte si svoje odpovede.**

## 2.6 Experimentovanie s parametrami

**Pripomenutie:** V kapitole 1 sme si vysvetlili, že v učení bez učiteľa často nevieme, koľko skupín existuje. Jedným z najdôležitejších parametrov K-means je počet skupín K. Experimentujme s rôznymi hodnotami a pozorujme, čo sa stane.

**Analógia:** Predstavte si, že máte oblečenie a neviete, koľko šuflíkov potrebujete. Skúšate 2 šuflíky, potom 3, potom 4... a pozorujete, ktoré rozdelenie dáva najväčší zmysel.

### 2.6.1 Vplyv počtu skupín

In [ ]:
# Experimentovanie s rôznym počtom skupín
# Skúšame K od 1 do 10 a pozorujeme, ako sa mení inertia
inertias = []
k_values = range(1, 11)

for k in k_values:
    kmeans_k = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_k.fit(X)
    inertias.append(kmeans_k.inertia_)  # Uložíme hodnotu inertia pre každé K

# Vykreslíme elbow plot
plt.figure(figsize=(14, 5))

plt.subplot(121)
plt.plot(k_values, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Počet skupín (K)')
plt.ylabel('Inertia (WCSS)')
plt.title('Elbow Metóda\n(nájdenie optimálneho počtu skupín)')
plt.axvline(x=3, color='r', linestyle='--', label='K=3 (skutočný počet)', linewidth=2)
plt.legend()
plt.grid(True, alpha=0.3)

# Ukážeme výsledky pre rôzne K
plt.subplot(122)
for k in [2, 3, 4, 5]:
    kmeans_k = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters_k = kmeans_k.fit_predict(X)
    plt.scatter(X[:, 2], X[:, 3], c=clusters_k, cmap='viridis', alpha=0.6, s=30, label=f'K={k}')
plt.xlabel('Petal Length (cm)')
plt.ylabel('Petal Width (cm)')
plt.title('Klastrovanie pre rôzne K')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Pozorovania:")
print("- Čím viac skupín, tým nižšia inertia")
print("- Ale viac skupín neznamená lepšie výsledky!")
print("- Elbow metóda: Hľadáme 'lakť' v grafe (kde sa zmenšovanie spomalí)")
print(f"- V našom prípade: K=3 vyzerá ako dobrá voľba (skutočný počet druhov)")

**Investigate:**

Pozrite sa na grafy:
- Čo sa stane s inertiou (vzdialenosťou), keď zväčšujeme počet skupín K?
- Kde je "lakť" v grafe? (kde sa zmenšovanie spomalí - ako lakť na ruke)
- **Dôležité pozorovanie:** Prečo viac skupín neznamená vždy lepšie výsledky?

**Jednoducho povedané:**
- Ak máme 1 skupinu, všetky kvety sú spolu - vzdialenosti sú veľké
- Ak máme 150 skupín (jeden kvet = jedna skupina), vzdialenosti sú nulové, ale to nedáva zmysel!
- Musíme nájsť správny počet skupín - nie príliš málo, nie príliš veľa

### 💭 Otázky na rozmyslenie (2.6.1)

1. Prečo klesá inertia (vzdialenosť), keď zväčšujeme počet skupín? (Pomôcka: Čím viac skupín, tým menej kvetov v každej skupine = kratšie vzdialenosti)
2. Čo znamená "lakť" v elbow grafe? (Pomôcka: Pozrite sa na graf - kde sa zmenšovanie spomalí? Je to ako lakť na ruke - kde sa línia "ohýba")
3. Prečo by sme nemali použiť veľmi veľa skupín? (Pomôcka: Čo by sa stalo, keby sme mali 150 skupín pre 150 kvetov? Dávalo by to zmysel?)

**Zapíšte si svoje odpovede.**

### 2.6.2 Vplyv náhodného začiatku (inicializácie)

**Pripomenutie:** V kroku 1 algoritmu sme náhodne vybrali začiatkové pozície stredov skupín. K-means je citlivý na tento náhodný začiatok. Pozrime sa, ako rôzne začiatky ovplyvňujú výsledky:

**Analógia:** Predstavte si, že máte mapu a označíte na nej 3 miesta. Ak začnete s rôznymi miestami, môžete skončiť s rôznymi skupinami. Preto algoritmus spustíme viackrát a vyberieme najlepší výsledok.

In [ ]:
# Porovnanie rôznych náhodných začiatkov (inicializácií)
# Každý graf ukazuje výsledok s iným náhodným začiatkom
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for seed in range(6):
    row = seed // 3
    col = seed % 3

    # n_init=1 znamená, že spustíme len raz (nie 10-krát)
    # init='random' znamená náhodný začiatok
    kmeans_seed = KMeans(n_clusters=3, random_state=seed, n_init=1, init='random')
    clusters_seed = kmeans_seed.fit_predict(X)
    centers_seed = kmeans_seed.cluster_centers_
    inertia_seed = kmeans_seed.inertia_  # Nižšia hodnota = lepší výsledok

    # Vykreslíme výsledky (zobrazíme len petal length vs petal width pre prehľadnosť)
    axes[row, col].scatter(X[:, 2], X[:, 3], c=clusters_seed, cmap='viridis', alpha=0.6, s=50)
    axes[row, col].scatter(centers_seed[:, 2], centers_seed[:, 3],
                          c='red', marker='X', s=200, linewidths=2)
    axes[row, col].set_title(f'Seed={seed}, Inertia={inertia_seed:.2f}')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Pozorovania:")
print("- Rôzne inicializácie vedú k rôznym výsledkom")
print("- Niektoré sú lepšie (nižšia inertia), niektoré horšie")
print("- Preto používame n_init=10 (spustí 10-krát a vyberie najlepší)")

### 💭 Otázky na rozmyslenie (2.6.2)

1. Prečo rôzne náhodné začiatky vedú k rôznym výsledkom? (Pomôcka: Pozrite sa na grafy - sú všetky rovnaké?)
2. Ktorá inicializácia je najlepšia? (Pomôcka: Pozrite sa na hodnoty inertia - čím nižšia, tým lepšie)
3. Prečo používame n_init=10? (Pomôcka: Ak spustíme algoritmus 10-krát s rôznymi začiatkami, môžeme vybrať najlepší výsledok)

**Zapíšte si svoje odpovede.**

## 2.7 Pokročilejšie vizualizácie a analýza

**Pripomenutie z kapitoly 1:** Máme 4 vlastnosti (dimenzie), ale môžeme vizualizovať len 2D alebo 3D. Preto vyberáme len niektoré vlastnosti alebo používame redukciu dimenzie (PCA, ktorú uvidíme detailne v kapitole 5).

### 2.7.1 3D vizualizácia

Máme 4 vlastnosti, ale môžeme vizualizovať len 3. Pozrime sa na 3D vizualizáciu (zobrazíme 3 vlastnosti naraz):

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# 3D vizualizácia (3 vlastnosti)
fig = plt.figure(figsize=(14, 6))

ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(X[:, 0], X[:, 1], X[:, 2], c=clusters, cmap='viridis', alpha=0.6, s=50)
ax1.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
           kmeans.cluster_centers_[:, 2], c='red', marker='X', s=200, linewidths=2)
ax1.set_xlabel('Sepal Length')
ax1.set_ylabel('Sepal Width')
ax1.set_zlabel('Petal Length')
ax1.set_title('3D Vizualizácia klastrov')

ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(X[:, 0], X[:, 1], X[:, 2],
           c=species_numeric, cmap='viridis', alpha=0.6, s=50)
ax2.set_xlabel('Sepal Length')
ax2.set_ylabel('Sepal Width')
ax2.set_zlabel('Petal Length')
ax2.set_title('3D Vizualizácia skutočných druhov')

plt.tight_layout()
plt.show()

### 2.7.2 Analýza vlastností skupín

**Jednoducho povedané:** Pozrime sa, aké sú priemerné hodnoty vlastností pre každú skupinu. To nám pomôže pochopiť, čo charakterizuje každú skupinu.

**Príklad:** Ak skupina 1 má priemernú dĺžku okvetného lístka 5 cm a skupina 2 má 2 cm, vieme, že skupina 1 obsahuje kvety s dlhšími okvetnými lístkami.

In [ ]:
# Analýza priemerných hodnôt pre každú skupinu
cluster_analysis = pl.DataFrame({
    'Skupina': range(3),
    'Sepal Length': [X[clusters == i, 0].mean() for i in range(3)],
    'Sepal Width': [X[clusters == i, 1].mean() for i in range(3)],
    'Petal Length': [X[clusters == i, 2].mean() for i in range(3)],
    'Petal Width': [X[clusters == i, 3].mean() for i in range(3)],
    'Počet kvetov': [np.sum(clusters == i) for i in range(3)]
})

print("Charakteristiky nájdených skupín:")
print(cluster_analysis)

# Porovnanie so skutočnými druhmi
species_analysis = df.group_by('species').agg([
    pl.col('sepal_length').mean().alias('sepal_length'),
    pl.col('sepal_width').mean().alias('sepal_width'),
    pl.col('petal_length').mean().alias('petal_length'),
    pl.col('petal_width').mean().alias('petal_width')
])
print("\nCharakteristiky skutočných druhov:")
print(species_analysis)

### 💭 Otázky na rozmyslenie (2.7.1)

1. Ktoré vlastnosti sú najdôležitejšie pre rozlíšenie skupín? (Pomôcka: Pozrite sa na priemerné hodnoty - kde sú najväčšie rozdiely?)
2. Ako sa líšia priemerné hodnoty medzi skupinami? (Pomôcka: Porovnajte hodnoty pre každú skupinu)
3. Ktorá skupina zodpovedá ktorému druhu? (Pomôcka: Porovnajte priemerné hodnoty skupín s priemernými hodnotami skutočných druhov)

**Zapíšte si svoje odpovede.**

## 2.8 Praktické úlohy (Modify a Make)

### 2.8.1 Úlohy Modify

**Úloha 1: Experimentovanie s počtom skupín**
- Skúste K-means s K=2, 4, 5, 10
- Vykreslite výsledky
- Porovnajte s K=3
- Odpovedzte: Ktoré K dáva najlepšie výsledky? Prečo?

**Úloha 2: Experimentovanie s inicializáciou**
- Skúste rôzne random_state hodnoty
- Porovnajte výsledky
- Odpovedzte: Prečo sú výsledky rôzne?

**Úloha 3: Analýza vlastností**
- Ktoré vlastnosti sú najdôležitejšie?
- Skúste klastrovanie len s 2 vlastnosťami (napr. petal length a petal width)
- Porovnajte výsledky s klastrovaním so všetkými 4 vlastnosťami

### 2.8.2 Úlohy Make

**Úloha 1: Aplikácia na vlastný dataset**
- Nájdite alebo vytvorte jednoduchý dataset (napr. výška a váha študentov)
- Aplikujte K-means
- Interpretujte výsledky

**Úloha 2: Porovnanie algoritmov**
- Skúste iný algoritmus klastrovania (napr. DBSCAN zo sklearn)
- Porovnajte výsledky s K-means
- Odpovedzte: Ktorý algoritmus funguje lepšie? Prečo?

**Zapíšte si všetky svoje odpovede, kódy a výsledky do samostatného dokumentu.**

## 2.9 Zhrnutie a závery

### 2.9.1 Kľúčové pojmy

- **Klastrovanie:** Automatické hľadanie skupín podobných objektov (ako triedenie oblečenia bez nápisov)
- **K-means:** Algoritmus klastrovania, ktorý hľadá K skupín pomocou stredov skupín
- **Centroid:** Stred skupiny (priemer bodov v skupine) - ako "ťažisko" skupiny
- **Inertia (WCSS):** Miera kvality klastrovania (čím nižšia, tým lepšie) - meria, ako blízko sú objekty k stredu skupiny
- **Elbow metóda:** Metóda na výber optimálneho počtu skupín - hľadáme "lakť" v grafe, kde sa zmenšovanie spomalí

### 2.9.2 Čo sme sa naučili

1. **Klastrovanie je typ učenia bez učiteľa**
   - Nemáme labely (ako pri triedení oblečenia bez nápisov)
   - Algoritmus automaticky nájde skupiny podobných objektov

2. **K-means opakuje niekoľko krokov**
   - Začneme s náhodnými miestami (stredy skupín)
   - Každý objekt sa presunie k najbližšiemu miestu
   - Vypočítame nové stredy skupín
   - Opakujeme, až kým sa nič neprestane meniť

3. **Vizualizácia je dôležitá**
   - Pomáha pochopiť výsledky (vidíme, či sú skupiny jasné)
   - Pomáha identifikovať problémy (vidíme chyby)

4. **Parametre sú dôležité**
   - Počet skupín K (musíme experimentovať, ak nevieme)
   - Náhodný začiatok (rôzne začiatky = rôzne výsledky)
   - Preto spúšťame algoritmus viackrát a vyberáme najlepší výsledok

### 2.9.3 Limitácie (kde K-means nefunguje dobre)

- **Musíme zadať počet skupín K** - v praxi často nevieme, koľko skupín existuje
- **Predpokladá kruhové skupiny** - nefunguje dobre na skupiny tvaru polmesiaca alebo iných komplikovaných tvarov
- **Citlivý na náhodný začiatok** - rôzne začiatky vedú k rôznym výsledkom (riešenie: spustiť viackrát)
- **Citlivý na nezvyčajné objekty** - nezvyčajné objekty môžu ovplyvniť pozíciu stredov skupín

### 2.9.4 Čo nás čaká ďalej

V kapitole 3 použijeme PyTorch na zložitejšie dáta (obrázky MNIST) a naučíme sa autoenkódery - neurónové siete, ktoré sa učia bez labelov.

### 💭 Otázky na rozmyslenie (2.9.1)

1. V akých situáciách by bolo užitočné použiť klastrovanie? (Pomôcka: Spomeňte si na príklady zo života, ktoré sme videli)
2. Aké sú výhody a nevýhody K-means algoritmu? (Pomôcka: Výhody = čo je dobré, nevýhody = limitácie, ktoré sme videli)
3. Ako by ste riešili problém, keď nevieme, koľko skupín existuje? (Pomôcka: Museli by ste experimentovať - ako?)

**Zapíšte si svoje odpovede.**

## 2.10 Zhrnutie úloh pre externé vypracovanie

V tejto kapitole ste mali vypracovať nasledujúce úlohy externe (mimo tohto notebooku). Toto je zhrnutie všetkých úloh a otázok na rozmyslenie, ktoré by ste mali mať vypracované:

---

### Otázky na rozmyslenie z jednotlivých podkapitol

#### Otázky z podkapitoly 2.1: Úvod do IRIS

1. Koľko vzorov máme? Koľko vlastností?
2. Aké vlastnosti máme? Čo predstavujú?
3. Ako sa líši tento dataset od MNIST?
4. Vidíte v grafoch nejaké skupiny? Koľko ich je?
5. Ktorá kombinácia vlastností najlepšie oddeluje skupiny?
6. Ako by sme zistili, koľko druhov existuje, keby sme nemali stĺpec species?

#### Otázky z podkapitoly 2.2: Čo je klastrovanie?

1. V akých situáciách by bolo užitočné použiť klastrovanie?
2. V čom sa líši klastrovanie od klasifikácie?
3. Prečo nemôžeme použiť klasifikáciu, keď nemáme labely?
4. Prečo je ľahšie nájsť skupiny v dobre oddelených dátach?
5. Ako by ste definovali "podobnosť" medzi dvoma kvetmi IRIS?

#### Otázky z podkapitoly 2.3: Ako funguje K-means?

1. Prečo sa centroidy menia v každej iterácii?
2. Čo sa stane, keď sa centroidy prestanú meniť?
3. Prečo je dôležité správne inicializovať centroidy?
4. Čo znamená "minimalizovať vzdialenosť"?
5. Prečo K-means nefunguje dobre na nekrehové skupiny?
6. Čo by ste robili, keby ste nevedeli, koľko skupín existuje?

#### Otázky z podkapitoly 2.4: Praktická aplikácia

1. Vidíte v grafoch jasné skupiny?
2. Ktorá kombinácia vlastností lepšie oddeluje skupiny?
3. Prečo sa skupiny v sepal grafoch viac prekrývajú?

#### Otázky z podkapitoly 2.5: Porovnanie a validácia

1. Ako dobre algoritmus "trafil" skutočné druhy?
2. Ktoré druhy boli najťažšie rozlíšiť? Prečo?
3. Čo to znamená, keď algoritmus nájde skupiny, ktoré nezodpovedajú presne skutočným druhom?
4. Prečo sa niektoré kvety zoskupili nesprávne?
5. Kde sa nachádzajú chyby v grafe?

#### Otázky z podkapitoly 2.6: Experimentovanie

1. Prečo klesá inertia, keď zväčšujeme počet skupín?
2. Čo znamená "lakť" v elbow grafe?
3. Prečo by sme nemali použiť veľmi veľa skupín?
4. Prečo rôzne inicializácie vedú k rôznym výsledkom?
5. Prečo používame n_init=10?

#### Otázky z podkapitoly 2.7: Pokročilejšie vizualizácie

1. Ktoré vlastnosti sú najdôležitejšie pre rozlíšenie skupín?
2. Ako sa líšia priemerné hodnoty medzi skupinami?
3. Ktorá skupina zodpovedá ktorému druhu?

#### Otázky z podkapitoly 2.9: Zhrnutie

1. V akých situáciách by bolo užitočné použiť klastrovanie?
2. Aké sú výhody a nevýhody K-means algoritmu?
3. Ako by ste riešili problém, keď nevieme, koľko skupín existuje?

---

### Praktické úlohy

#### Úlohy Modify (2.8.1)

**Úloha 1: Experimentovanie s počtom skupín**
- Kód s K=2, 4, 5, 10
- Grafy výsledkov
- Porovnanie s K=3
- Odpovede na otázky

**Úloha 2: Experimentovanie s inicializáciou**
- Kód s rôznymi random_state
- Porovnanie výsledkov
- Odpovede na otázky

**Úloha 3: Analýza vlastností**
- Kód s klastrovaním len 2 vlastností
- Porovnanie so 4 vlastnosťami
- Odpovede na otázky

#### Úlohy Make (2.8.2)

**Úloha 1: Aplikácia na vlastný dataset**
- Kód aplikácie K-means
- Interpretácia výsledkov

**Úloha 2: Porovnanie algoritmov**
- Kód s DBSCAN (alebo iným algoritmom)
- Porovnanie s K-means
- Odpovede na otázky

---

### Formát odovzdania

Všetky úlohy a odpovede na otázky by mali byť vypracované v **samostatnom dokumente (iba PDF)** s nasledujúcou štruktúrou:

#### Štruktúra dokumentu:

1. **Otázky na rozmyslenie z jednotlivých podkapitol**
   - Sekcia pre každú podkapitolu (2.1 až 2.9)
   - Odpovede na všetky otázky z každej sekcie
   - Vlastné úvahy a pozorovania

2. **Praktické úlohy**
   - Úlohy Modify (2.8.1)
   - Úlohy Make (2.8.2)

3. **Pre každú praktickú úlohu:**
   - **Názov úlohy**
   - **Kód** (ak je súčasťou úlohy) - môže byť v samostatnom `.py` súbore
   - **Výsledky** - obrázky grafov, výstupy z kódu, tabuľky
   - **Odpovede na otázky** - stručné, ale úplné odpovede
   - **Pozorovania a závery** - čo ste sa naučili, aké problémy ste mali

---

### Hodnotenie

Úlohy budú hodnotené podľa:
- **Úplnosti** - všetky otázky a úlohy sú vypracované
- **Hĺbky úvahy** - nielen správne odpovede, ale aj premýšľanie nad problémom
- **Správnosti** - správne odpovede a funkčný kód (kde je to relevantné)
- **Kvality kódu** - čitateľný, dobre komentovaný kód (ak je súčasťou úlohy)
- **Kritického myslenia** - schopnosť analyzovať, hodnotiť a aplikovať koncepty

---

### Dátum odovzdania

**Dátum odovzdania:** Podľa pokynov vášho učiteľa

**Formát súborov:**
- PDF dokument: `Kapitola2_Ulohy_VaseMeno.pdf`
- Python súbory: `uloha_2_8_1.py`, `uloha_2_8_2.py`, atď.